In [1]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 10


In [2]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [3]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules
using PyCall
pd = pyimport("pandas")

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "spring.jl"))
    includef(srcdir("scenarios", "fall.jl"))
end

┌ Info: Precompiling BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455]
└ @ Base loading.jl:1317
┌ Info: Skipping precompilation since __precompile__(false). Importing BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455].
└ @ Base loading.jl:1025
┌ Warning: skipping callee #_#38 (called by nothing) due to UndefRefError()
└ @ LoweredCodeUtils /afs/inf.ed.ac.uk/user/s16/s1672897/.julia/packages/LoweredCodeUtils/poBmh/src/signatures.jl:279


Report results on FALL

In [4]:
function report_phys101(dataset; verbose=false)
    dataset = "phys101/$dataset"
    df = pd.DataFrame(
        columns=("Method", "Normliased RMSE (train)", "Normliased RMSE (test)", "EM seed", "Data seed")
    )

    count = 1
    for seed in 1:3, shuffleseed in 1:6
        hps = (ntrains=2, seed=seed, shuffleseed=shuffleseed)

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 2; shuffleseed=hps.shuffleseed, istrain=false)

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "em.jld2"))
        catch
            println("seed $(hps.seed) unfinished")
        end

        @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, trace = res

        for (force, method) in [(trace[end].force, "BSP"), (ZeroForce(), "Zero")]
            verbose && println("count: $count")
            verbose && if method == "BSP"
                println(BayesianSymbolic.get_executable(force.tree, force.grammar))
                println(force.constant)
            end
            evalres = evalforce(ScenarioModel, scenarios, attributes, force, ealg, elike, mlike)
            evalres_test = evalforce(ScenarioModel, scenarios_test, attributes_test, force, ealg, elike, mlike)

            # Add to data frame
            df.loc[count] = [method, evalres.normrmse, evalres_test.normrmse, seed, shuffleseed]
            count = count + 1
        end
        verbose && println()
    end
    
    return df
end

report_phys101 (generic function with 1 method)

Report results on FALL

In [5]:
df = report_phys101("fall"; verbose=true)

count: 1
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.26566997688108196, 0.0, 0.0]
count: 2

count: 3
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.2797620450420885, 0.0, 0.0]
count: 4

count: 5
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.28389818842924985, 0.0, 0.0]
count: 6

count: 7
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.24720636071342483, 0.0, 0.0]
count: 8

count: 9
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.2513425041005862, 0.0, 0.0]
count: 10

count: 11
c1 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[-0.2655051936474395, 0.0, 0.0]
count: 12

count: 13
c2 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[0.0, -0.26566997688108196, 0.0]
count: 14

count: 15
c2 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[0.0, -0.2797620450420885, 0.0]
count: 16

count: 17
c2 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[0.0, -0.28389818842924985, 0.0]
count: 18

count: 19
c2 * bmul(bmul(mi, mj), bnormalize(bsub(pi, pj)))
[0.0, -0.24720636

,Method,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
0,BSP,0.050459,0.048295,1,1
1,Zero,0.054886,0.046081,1,1
2,BSP,0.050874,0.047763,1,2
3,Zero,0.050874,0.050094,1,2
4,BSP,0.026550,0.072051,1,3
5,Zero,0.028916,0.072051,1,3
6,BSP,0.072051,0.026856,1,4
7,Zero,0.072051,0.028916,1,4
8,BSP,0.047999,0.050874,1,5
9,Zero,0.050094,0.050874,1,5


In [6]:
df.groupby(["Method"]).median()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.049376,0.049378
Zero,0.050484,0.050484


In [7]:
df.groupby(["Method"]).quantile([.25, .75])

PyObject              Normliased RMSE (train)  Normliased RMSE (test)
Method                                                      
BSP    0.25                 0.047999                0.047763
       0.75                 0.050874                0.050874
Zero   0.25                 0.046081                0.046081
       0.75                 0.054886                0.054886

In [8]:
df.groupby(["Method"]).mean()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.049371,0.049383
Zero,0.050484,0.050484


In [9]:
df.groupby(["Method"]).std()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.013558,0.013472
Zero,0.013079,0.013079


Report results on SPRING

In [10]:
df = report_phys101("spring"; verbose=true)

count: 1
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.9000251775164474, 0.0, 0.0]
count: 2

count: 3
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.9370444166995548, 0.0, 0.0]
count: 4

count: 5
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.9704398527767122, 0.0, 0.0]
count: 6

count: 7
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.946621327233317, 0.0, 0.0]
count: 8

count: 9
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.9470436833884384, 0.0, 0.0]
count: 10

count: 11
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bnormalize(bsub(pi, pj)))
[-1.9759082472507148, 0.0, 0.0]
count: 12

count: 13
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bdiv(bsub(pi, pj), bnorm(bsub(pi, c))))
[-1.8753900200652844, 0.0, 0.0]
count: 14

count: 15
c1 * bmul(bsub(bnorm(bsub(pi, c)), lj), bdiv(bsub(pi, pj), bnorm(bsub(pi, c))))
[-1.9399628120534058, 0.0, 0.0]
count: 16

count: 17
c1 * bmul(bsub(bnorm

,Method,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
0,BSP,0.009406,0.009489,1,1
1,Zero,0.020954,0.022814,1,1
2,BSP,0.008500,0.010215,1,2
3,Zero,0.021907,0.021862,1,2
4,BSP,0.010629,0.008419,1,3
5,Zero,0.022095,0.021673,1,3
6,BSP,0.008528,0.010157,1,4
7,Zero,0.021673,0.022095,1,4
8,BSP,0.010538,0.008588,1,5
9,Zero,0.021862,0.021907,1,5


In [11]:
df.groupby(["Method"]).median()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.009107,0.009851
Zero,0.021884,0.021884


In [12]:
df.groupby(["Method"]).quantile([.25, .75])

PyObject              Normliased RMSE (train)  Normliased RMSE (test)
Method                                                      
BSP    0.25                 0.008813                0.008757
       0.75                 0.010271                0.010148
Zero   0.25                 0.021673                0.021673
       0.75                 0.022095                0.022095

In [13]:
df.groupby(["Method"]).mean()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.009450,0.009632
Zero,0.021884,0.021884


In [14]:
df.groupby(["Method"]).std()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.000790,0.000886
Zero,0.000567,0.000567
